<a href="https://colab.research.google.com/github/AlexH20/Supervised-ML-sentiment-measures/blob/main/Get_paneldata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wrds
!pip install holidays
!pip install pandas_market_calendars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 13.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 7.0 MB/s 
     |████████████████████████████████| 152 kB 33.8 MB/s 
  Created wheel for exchange-calendars: filename=exchange_calendars-3.6.3-py3-none-any.whl size=182636 sha256=759dc5f032624d9b9fd659cdbf797a73b84709b0d98bc4435ae83af59b8573c2
  Stored in directory: /root/.cache/pip/wheels/b8/a3/19/b4611514d34ffd61d13aef10fefc2dcaf3754145121ceba647
Successfully built exchange-calendars


In [ ]:
import wrds
import csv
from datetime import datetime, timedelta, date
import sys
import statsmodels.formula.api as sm
import pandas as pd
from google.colab import drive 
import holidays
import pandas_market_calendars as mcal
drive.mount("/content/gdrive")

#To max out field limit
csv.field_size_limit(sys.maxsize)

db = wrds.Connection(wrds_username='holzeral')

# Function to check whether date is on a non-trading day. If so, then return next trading day.

holidaysUS = holidays.US()
nyse = mcal.get_calendar('NYSE')
stock_holidays = nyse.holidays()

stock_holidays = list(pd.to_datetime(stock_holidays.holidays))
stock_holidays = [x.date() for x in stock_holidays]

#Excluded: GLD, SPX, DAX, USD, QQQ, SQ, SNAP, FIT, JCP, FRA, SHOP, AA, PYPL, VRX, ABX, UTX, SPY, AGN, CELG, APC

tickers = ['AAPL', 'AMZN', 'TSLA', 'FB', 'BA', 'NFLX', 'DIS', 'EFX', 'BAC', 'INTC', 'F', 'GE', 'GM', 
 'MSFT', 'SBUX', 'AIR', 'AAL', 'IBM', 'JPM', 'CMG', 'WFC', 'C', 'TWTR', 'WMT', 'MCD', 'AMD', 'NVDA', 'JNJ', 'GS', 
 'BABA', 'CAT', 'MU', 'CSCO', 'XOM', 'CVX', 'BP', 'GOOGL', 'GPRO', 'COST', 'HD', 'NKE', 'KO', 
 'AXP', 'TGT', 'ATVI', 'CMCSA', 'DAL', 'LMT', 'T', 'ABBV', 'PFE', 'GILD', 'ADBE', 'CRM', 'VZ', 'AVGO', 'BX', 
 'LULU', 'BLK', 'UNH', 'KMI', 'BBY', 'PG', 'AGI', 'AMAT', 'MRK', 'M', 'BIDU', 'QCOM', 'FDX', 'AMGN', 
 'BMY', 'ORCL', 'BHP', 'MA', 'KR', 'MO', 'GME', 'PM', 'CHK', 'MMM', 'BBBY', 'COP', 'IRBT', 'MS', 
 'FCX', 'HAL', 'HPQ', 'UAL','JWN', 'CVS', 'V', 'EA', 'STZ', 'GLW', 'ADP', 'AZN', "EBAY", 'ACN', 'PEP']


def check_trading_dayandhour(day):
    trading_day = day
    while trading_day.weekday() in holidays.WEEKEND or trading_day in stock_holidays:
        trading_day += timedelta(1)
    return trading_day

#Output path
output_file_path = "gdrive/My Drive/Thesis/processed data/"
output_file_name = "data_whole_woScAR.csv"

#Get all dates between 2015 and 2019

start_date = date(2015, 1, 1)
end_date = date(2019, 12, 31)
delta = end_date - start_date
dates = []

for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    dates.append(day)

#List of 100 most frequent stock ticker symbols in NASDAQ news data set between 2015 and 2019

data_fill = []
i = 0

for tick in tickers[:25]:

    for date in dates:

        date_adj = check_trading_dayandhour(date)

        ########## Following code uses wrds database to extract data for control variable: share turnover and nasdaq dummy ##########

        # Substract days from datetime variable to get start and end date in API request
        day0 = date_adj - timedelta(0)
        dayminus1 = date_adj - timedelta(1)
        dayminus252 = date_adj - timedelta(252)
        dayminus6 = date_adj - timedelta(6)

        # Convert to proper time format for wrds request
        day0 = day0.strftime("%m/%d/%Y")
        dayminus1 = dayminus1.strftime("%m/%d/%Y")
        dayminus6 = dayminus6.strftime("%m/%d/%Y")
        dayminus252 = dayminus252.strftime("%m/%d/%Y")

        permno = db.raw_sql("""select permno, date
                                                        from crsp.dse
                                                        where TICKER = '{}'
                                                        and date  <= '{}'""".format(tick, day0))
        

        #Get latest permno
        permno["date"] = pd.to_datetime(permno["date"], format = "%Y/%m/%d")
        permno = permno.sort_values(by = "date")
        permno = permno["permno"].iloc[-1]

        # data query to get control variable share turnover: volume within (-252,-6) divided by shares outstanding on 0
        voldata = db.raw_sql("""select vol 
                                                    from crsp.dsf
                                                    where permno = {}
                                                    and date between '{}' and '{}'""".format(permno, dayminus252,
                                                                                             dayminus6))

        shrout_m1 = db.raw_sql("""select shrout, date
                                                    from crsp.dsf
                                                    where permno = {}
                                                    and date between '{}' and '{}'""".format(permno, dayminus6, day0))
        
        issuno = db.raw_sql("""select issuno
                                                    from crsp.dsf
                                                    where permno = {}
                                                    and date between '{}' and '{}'""".format(permno, dayminus252, day0))    
                                            
        if int(issuno["issuno"].iloc[-1]) != 0:
            issuno_i = 1
        else:
            issuno_i = 0

        shareturnover = (int(voldata["vol"].sum()) / (int(shrout_m1["shrout"].iloc[-1]) * 1000))

        ########## Following code uses wrds database to extract data for control variable: size  ##########

        # Size defined as price * shrout on date -1
        price_and_shroutm1 = db.raw_sql("""select prc, shrout
                                                            from crsp.dsf
                                                            where permno = {}
                                                            and date between '{}' and '{}'""".format(permno, dayminus1,day0))
        

        size = price_and_shroutm1["prc"][0] * int(price_and_shroutm1["shrout"][0])


        ########## Following code uses wrds database to extract data for control variable: BTM  ##########

        dayplus5years = date + timedelta(1825)
        dayplus5years = dayplus5years.strftime("%m/%d/%Y")

        assets_and_equity = db.raw_sql("""select tic, datadate, seq
                                                            from comp.funda
                                                            where tic = '{}'
                                                            and datadate between '{}' and '{}'""".format(tick,
                                                                                                         day0,
                                                                                                         dayplus5years))
        assets_and_equity = assets_and_equity.dropna()

        # BTM defined as book value of total stockholder equity divided by MVE

        btm = (assets_and_equity["seq"].iloc[0] / (size / 1000))

        ########## Following code uses wrds database to extract data for control variable: pref_alpha  ##########

        ret = db.raw_sql("""select date, ret 
                                                                        from crsp.dsf
                                                                        where permno = {}
                                                                        and date between '{}' and '{}'""".format(permno,
                                                                                                                 dayminus252,
                                                                                                                 dayminus6))

        ff = db.raw_sql("""select date, mktrf, smb, hml, rf
                                                                        from ff.factors_daily
                                                                        where date between '{}' and '{}'""".format(
            dayminus252,
            dayminus6))

        # Change fetched data to pandas dataframe
        ff = pd.DataFrame(ff, columns=["date", "mktrf", "smb", "hml", "rf"])
        ret = pd.DataFrame(ret, columns=["date", "ret"])

        # Merge the two dataframes on date and drop NAs so only dates with no missing values are included in the regression
        data_reg = ret.merge(ff,
                             on="date",
                             how="left")

        data_reg = data_reg[data_reg.notna()]

        # Substract risk free rate from daily return to get excess return
        data_reg["excess_return"] = data_reg["ret"] - data_reg["rf"]
        data_reg = data_reg.drop(["rf", "date"], axis=1)

        if (len(data_reg)) >= 60:
          model = sm.ols("excess_return ~ mktrf + smb + hml", data=data_reg).fit()
          pre_falpha = model.params["Intercept"]
        else:
          raise ValueError("Data incomplete")

        ########## Following code uses wrds database to extract data for dependent variable: excess returns  ##########
        # Excess return defined as 1 day period buy and hold return from day -1 to day 1 minus cumul value weighted CRSP return in same period


        # Fetch prices and returns for given buy and hold period
        exc_ret = db.raw_sql("""select date, prc, ret
                                                            from crsp.dsf
                                                            where permno = {}
                                                            and date = '{}'""".format(permno,day0))

        vw_ret = db.raw_sql("""select date, vwretd
                                                            from crsp.dsi
                                                            where date = '{}'""".format(day0))
        

        # Get abnormal returns

        return_i = exc_ret["ret"][0]
        vw_return_i = vw_ret["vwretd"][0]

        ar = return_i - vw_return_i

        # Also add normal return for possible FF3 regression.

        data_fill.append([date, tick, issuno_i, shareturnover,
                      size, btm, pre_falpha,
                      ar, return_i])
        
        i += 1
        print(i)

df = pd.DataFrame(data_fill,
                  columns=["Date", "Ticker", "Nasdaq",
                           "Turnover",
                           "Size", "BTM", "pref_alpha", "AR", "Return"])

with open(output_file_path + output_file_name, "w") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(
        ["Date", "Ticker", "Nasdaq", "Turnover", "Size", "BTM",
         "pref_alpha", "AR", "Return"])
    for index, row in df.iterrows():
        writer.writerow(row)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Enter your WRDS username [holzeral]:holzeral
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
Loading library list...
Done
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 77))



KeyboardInterrupt: ignored